In [1]:
import numpy as np

# Load tudatpy modules
from tudatpy.interface import spice
from tudatpy.util import result2array
from tudatpy import numerical_simulation
from utils import get_form_initial_conditions
from tudatpy.astro.time_conversion import DateTime
from tudatpy.numerical_simulation import environment_setup, propagation_setup, estimation_setup

In [2]:
# Load spice kernels
spice.load_standard_kernels()

# Set simulation start (0 TDB seconds since J2000) and end epochs
dt = 60.0
T = 360
simulation_start_epoch = DateTime(2000, 1, 1).epoch()
simulation_end_epoch = (T - 1) * dt

In [10]:
# Create default body settings for "Venus", "Moon", "Earth", "Mars", "Jupiter", "Sun"
bodies_to_create = ["Venus", "Moon", "Earth", "Mars", "Jupiter", "Sun"]

# Create default body settings for bodies_to_create, with "Earth"/"J2000" as the global frame origin and orientation
global_frame_origin = "Earth"
global_frame_orientation = "J2000"
body_settings = environment_setup.get_default_body_settings(bodies_to_create, global_frame_origin, global_frame_orientation)
body_settings.get("Earth").atmosphere_settings = environment_setup.atmosphere.nrlmsise00()

# Create system of bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

In [11]:
# Create vehicle objects.
spacecrafts = ["Chief", "Deputy1", "Deputy2", "Deputy3"]
for spacecraft in spacecrafts:
    bodies.create_empty_body(spacecraft)

# Set mass of satellites
for spacecraft in spacecrafts:
    bodies.get(spacecraft).mass = 1.0

# Create aerodynamic coefficient interface settings
reference_area = 1e-2
drag_coefficient = 2.22
aero_coefficient_settings = environment_setup.aerodynamic_coefficients.constant(reference_area, [drag_coefficient, 0.0, 0.0])

# Add the aerodynamic interface to the environment
for spacecraft in spacecrafts:
    environment_setup.add_aerodynamic_coefficient_interface(bodies, spacecraft, aero_coefficient_settings)

In [12]:
# Define bodies that are propagated
bodies_to_propagate = ["Chief"]#, "Deputy1", "Deputy2", "Deputy3"]

# Define central bodies of propagation
central_bodies = ["Earth"]#, "Earth", "Earth", "Earth"]

In [13]:
# Define radiation pressure settings
reference_area_radiation = reference_area
radiation_pressure_coefficient = 1.2
occulting_bodies_dict = dict()
occulting_bodies_dict["Sun"] = ["Moon", "Earth"]

# Create and add the radiation pressure interface to the environment
for spacecraft in spacecrafts:
    radiation_pressure_settings = environment_setup.radiation_pressure.cannonball(
        "Sun", reference_area_radiation, radiation_pressure_coefficient, occulting_bodies=occulting_bodies_dict["Sun"]
    )
    environment_setup.add_radiation_pressure_interface(
        bodies, spacecraft, radiation_pressure_settings
    )
# https://docs.tudat.space/en/latest/_src_user_guide/state_propagation/propagation_setup/translational/radiation_pressure_acceleration.html#backwards-compatibility
    
# Define accelerations acting on each vehicle
accelerations_settings = dict(
    Venus=[propagation_setup.acceleration.point_mass_gravity()],
    Moon=[propagation_setup.acceleration.point_mass_gravity()],
    Earth=[propagation_setup.acceleration.spherical_harmonic_gravity(12, 12), propagation_setup.acceleration.aerodynamic()],
    Mars=[propagation_setup.acceleration.point_mass_gravity()],
    Jupiter=[propagation_setup.acceleration.point_mass_gravity()],
    Sun=[propagation_setup.acceleration.point_mass_gravity(), propagation_setup.acceleration.cannonball_radiation_pressure()]
)

# Create global accelerations settings dictionary
acceleration_settings = {"Chief": accelerations_settings}#,
                        #  "Deputy1": accelerations_settings,
                        #  "Deputy2": accelerations_settings,
                        #  "Deputy3": accelerations_settings}
                         

# Create acceleration models
acceleration_models = propagation_setup.create_acceleration_models(
    bodies,
    acceleration_settings,
    bodies_to_propagate,
    central_bodies)

In [14]:
# Create initial state
initial_states = get_form_initial_conditions(1)[:6]
print(initial_states)

# Create termination settings
termination_condition = propagation_setup.propagator.time_termination(simulation_end_epoch)

# Create numerical integrator settings
fixed_step_size = dt
integrator_settings = propagation_setup.integrator.runge_kutta_4(fixed_step_size)

# Create propagation settings
propagator_settings = propagation_setup.propagator.translational(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    initial_states,
    simulation_start_epoch,
    integrator_settings,
    termination_condition
)

# Setup parameters settings to propagate the state transition matrix
parameter_settings = estimation_setup.parameter.initial_states(propagator_settings, bodies)

# Create the parameters that will be estimated
parameters_to_estimate = estimation_setup.create_parameter_set(parameter_settings, bodies)

# Create the variational equation solver and propagate the dynamics
variational_equations_solver = numerical_simulation.create_variational_equations_solver(
    bodies, propagator_settings, parameters_to_estimate, simulate_dynamics_on_creation=True
)

# Extract the resulting state history, state transition matrix history, and sensitivity matrix history
states = variational_equations_solver.state_history
state_transition_matrices = variational_equations_solver.state_transition_matrix_history
X_tudat = result2array(states)[:, 1:]

[[ 6.89590615e+06]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-9.91593062e+02]
 [ 7.54206873e+03]]


In [15]:
# Create default body settings for "Earth"
bodies_to_create = ["Earth"]
global_frame_origin = "Earth"
global_frame_orientation = "J2000"
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation)
# define parameters of an invariant exponential atmosphere model
scale_height = 7.249e3
surface_density = 1.225
# create atmosphere settings and add to body settings of "Earth"
body_settings.get( "Earth" ).atmosphere_settings = environment_setup.atmosphere.exponential(scale_height, surface_density)
bodies = environment_setup.create_system_of_bodies(body_settings)
bodies.create_empty_body("Chief")
bodies.get("Chief").mass = 1.0
reference_area = 1e-2
drag_coefficient = 2.22
aero_coefficient_settings = environment_setup.aerodynamic_coefficients.constant(
    reference_area, [drag_coefficient, 0.0, 0.0]
)
environment_setup.add_aerodynamic_coefficient_interface(
            bodies, "Chief", aero_coefficient_settings)
bodies_to_propagate = ["Chief"]
central_bodies = ["Earth"]
accelerations_settings = dict(
    Earth=[
        propagation_setup.acceleration.spherical_harmonic_gravity(2, 0),
        propagation_setup.acceleration.aerodynamic()
    ])
acceleration_settings = {"Chief": accelerations_settings}
                         
acceleration_models = propagation_setup.create_acceleration_models(
    bodies,
    acceleration_settings,
    bodies_to_propagate,
    central_bodies)
fixed_step_size = dt
termination_condition = propagation_setup.propagator.time_termination(simulation_end_epoch)
integrator_settings = propagation_setup.integrator.runge_kutta_4(fixed_step_size)

X_mine = np.zeros((T, 6))
X_mine[0, :] = initial_states.reshape(6)
for t, X_initial in enumerate(X_tudat):
    if t < T - 1:
        propagator_settings = propagation_setup.propagator.translational(
            central_bodies,
            acceleration_models,
            bodies_to_propagate,
            X_initial,
            simulation_start_epoch,
            integrator_settings,
            termination_condition
        )
        parameter_settings = estimation_setup.parameter.initial_states(propagator_settings, bodies)
        parameters_to_estimate = estimation_setup.create_parameter_set(parameter_settings, bodies)
        variational_equations_solver = numerical_simulation.create_variational_equations_solver(
            bodies, propagator_settings, parameters_to_estimate, simulate_dynamics_on_creation=True
        )
        states = variational_equations_solver.state_history
        X_mine[t + 1, :] = result2array(states)[1, 1:]
        
# for t in range(T):
#     print(X_tudat[t, :3], X_mine[t, :3], X_tudat[t, :3] - X_mine[t, :3])
#     print()

# Compute the difference between the ground truth and the model
diff = X_tudat[1:, :] - X_mine[1:, :]

# The new shape will be (T * 24, 1) where each column represents a different state variable
diff_flattened = diff.reshape(-1, 6)

# Calculate the general covariance matrix of the flattened differences
covariance_matrix = np.cov(diff_flattened, rowvar=False)
print(covariance_matrix[:3, :3])
print()
print(covariance_matrix[:3, 3:])
print()
print(covariance_matrix[3:, :3])
print()
print(covariance_matrix[3:, 3:])

[[ 2.04920688e-12 -6.27351042e-15  2.76594548e-14]
 [-6.27351042e-15  6.51172857e-14 -2.95620693e-13]
 [ 2.76594548e-14 -2.95620693e-13  1.96545798e-12]]

[[ 6.82574464e-14 -4.78647011e-16  3.06735124e-15]
 [ 4.93166655e-17  2.16897757e-15 -9.84973833e-15]
 [-1.15607695e-15 -9.84326888e-15  6.54463873e-14]]

[[ 6.82574464e-14  4.93166655e-17 -1.15607695e-15]
 [-4.78647011e-16  2.16897757e-15 -9.84326888e-15]
 [ 3.06735124e-15 -9.84973833e-15  6.54463873e-14]]

[[ 2.27594115e-15 -7.34385803e-18  3.30209947e-17]
 [-7.34385803e-18  7.22929004e-17 -3.28291355e-16]
 [ 3.30209947e-17 -3.28291355e-16  2.18165989e-15]]


In [16]:
for t in range(T):
    print(X_tudat[t, :3], X_mine[t, :3], X_tudat[t, :3] - X_mine[t, :3])
    print()

[6895906.15       0.         0.  ] [6895906.15       0.         0.  ] [0. 0. 0.]

[6880802.91407968  -59452.13258509  452192.71741932] [6880802.91407968  -59452.13258509  452192.71741932] [0. 0. 0.]

[6835560.21096672 -118643.85362935  902399.25202788] [6835560.21096671 -118643.85362935  902399.25202789] [ 2.79396772e-09 -4.36557457e-11 -7.33416528e-09]

[6760378.70214912 -177315.93566659 1348642.47476104] [6760378.70214911 -177315.93566659 1348642.47476105] [ 9.31322575e-09 -2.61934474e-10 -1.35041773e-08]

[6655591.73103709 -235211.51198129 1788963.30826998] [6655591.73103707 -235211.51198129 1788963.30827   ] [ 1.86264515e-08 -6.98491931e-10 -1.74622983e-08]

[6521663.69400514 -292077.23974506 2221429.62085332] [6521663.69400511 -292077.23974506 2221429.62085334] [ 3.16649675e-08 -1.51339918e-09 -1.76951289e-08]

[6359187.77880818 -347664.44273535 2644144.96412925] [6359187.77880813 -347664.44273535 2644144.96412927] [ 4.47034836e-08 -2.79396772e-09 -1.49011612e-08]

[6168883.088816